# Understanding PLONK (Part 3): Permutation Argument

The Plonkish circuit encoding uses two matrices, $(Q,\sigma)$, to describe the blank structure of the circuit. Here, $Q$ represents the computation switches, and $\sigma$ defines the permutation constraints, ensuring that certain positions in the $W$ matrix must be filled with equal values. This article focuses on explaining the principles of permutation argument.

## Revisiting Copy Constraints
Recall the Plonkish $W$ table, which has three columns:

$$
\begin{array}{c|c|c|c|}
i & w_{a,i} & w_{b,i} & w_{c,i}  \\
\hline
1 & {\color{red}x_6} & {\color{blue}x_5} & {\color{green}out} \\
2 & x_1 & x_2 & {\color{red}x_6} \\
3 & x_3 & x_4 & {\color{blue}x_5} \\
4 & 0 & 0 & {\color{green}out} \\
\end{array}
$$

We want to constrain the Prover to fill in the $W$ table such that the following copy relationships are satisfied: $w_{a,1}=w_{c,2}$, $w_{b,1}=w_{c,3}$​, and $$w_{c,1}=w_{c,4}$. In other words, the value in the $w_{a,1}$​ position needs to be copied to the $w_{c,2}$​ position, the value in the $w_{b,1}$​ position needs to be copied to the $w_{c,3}$​ position, and the value in the $w_{c,1}$​ position needs to be copied to the $w_{c_4}$​ position.

The challenge lies in the fact that the Verifier must verify multiple copy relationships in the $W$ table with only one random challenge, and without seeing the $W$ table.

Plonk's "copy constraints" are implemented through the permutation argument, which involves cyclically permuting the values in the table that need to be constrained to be equal, and then proving that the permuted table is identical to the original table.

To simplify the problem: how do we prove that two vectors of equal length $\vec{a}$ and $\vec{a}'$ satisfy a known permutation $\sigma$, and that $\vec{a}=\vec{a}'$:

$$
a_i=a'_{\sigma(i)}
$$

In the following sections, we will reduce the permutation argument --> multiset equality --> grand product argument.

## Permutation Argument --> Multiset Equality

We can express the permutation constraint as an instance of a multiset equality constraint, i.e. a check that two multisets are equal. (A multiset is a modification of the concept of a set that, unlike a set, allows for duplicates of each of its elements.)

For illustration, suppose we wanted to prove that two vectors satisfy an odd-even index swap permutation:

$$
\begin{array}{rcl}
\vec{a} &=& (a_0, a_1, a_2, a_3,\ldots, a_{n-1}, a_n) \\
\vec{b} &=& (a_1, a_0, a_3, a_2, \ldots, a_n, a_{n-1})\\
\end{array}
$$

Now, we introduce the following "position vectors" to specify the odd-even swap:

$$
\vec{i}=(0,1, 2,3,\ldots, n-1, n),\quad \sigma = (1, 0, 3, 2,\ldots, n, n-1)
$$

We can then align these position vectors with $\vec{a}$ and $\vec{b}$:

$$
\begin{array}{|c|c | c|c|}
a_i & {i} & b_i & \sigma({i}) \\
\hline
a_0 & 0 & b_0=a_1 & 1 \\
a_1 & 1 & b_1=a_0 & 0 \\
a_2 & 2 & b_2=a_3 & 3 \\
a_3 & 3 & b_3=a_2 & 2 \\
\vdots & \vdots & \vdots & \vdots \\
a_n & n & b_n=a_{n-1} & n-1 \\
a_{n-1} & n-1 & b_{n-1}=a_{n} & n \\
\end{array}
$$

In effect, each element in the position vector labels a value in the original column; we then check for equality between multisets of tuples $\{a'_i = (a_i, i)\}$ and $\{b'_i = (b_1, \sigma(i))\}$.

$$
\begin{array}{|c|c|}
a'_i=(a_i, i) & b'_i=({b}_i, \sigma(i)) \\
\hline
(a_0, 0) & (b_0=a_1, 1) \\
(a_1, 1) & (b_1=a_0, 0) \\
\vdots & \vdots \\
(a\_{n-1}, n-1) & (b\_{n-1}=a\_{n}, n) \\
(a\_n, n) & (b\_n=a\_{n-1}, n-1) \\
\end{array}
$$

It's easy to see that if two vectors $\vec{a}$ and $\vec{b}$ satisfy the $\sigma$ permutation, then the indexed vectors $\vec{a}'$ and $\vec{b}'$ will be equal as multisets.

Lastly, to encode each tuple as a field element, we request another random challenge $\beta$ from the Verifier:

$$
\begin{array}{|c|c|}
a'_i=(a_i+\beta\cdot i) & b_i'=(b + \beta\cdot \sigma(i)) \\
\hline
(a_0 + \beta\cdot 0) & (b_0 + \beta\cdot 1) \\
(a_1 + \beta\cdot 1) & (b_1 + \beta\cdot 0) \\
\vdots & \vdots \\
(a_{n-1} + \beta\cdot n-1) & (b_{n-1} + \beta\cdot n) \\
(a_n + \beta\cdot n) & (b_n + \beta\cdot (n-1))\\
\end{array}
$$

In order to express a multiset equality check $\vec{a'} = \vec{b'}$ as a polynomial identity, we can encode each vector as the roots of a polynomial:

\begin{align*}
a'(X) &:= \prod (X - a'_i) = (X - a'_0)(X - a'_1) \ldots (X - a'_{n-1}) \\
b'(X) &:= \prod (X - b'_i) = (X - b'_0)(X - b'_1) \ldots (X - b'_{n-1}) \\
\end{align*}

If the two polynomials $a'(X), b'(X)$ have the same roots, then they must be equal. By invoking the Schwartz-Zippel lemma as before, we can check equality of $a'(X), b'(X)$ by evaluating them at a random verifier challenge $\gamma$:

\begin{align*}
&\prod (\gamma - a'_i) \overset{?}{=} \prod (\gamma - b'_i) \\
\end{align*}

## Multiset Equality --> Grand Product Argument

As written, $a'(X), b'(X)$ are both degree-$n$ polynomials, which are too large to work with. In order to express this identity as a low-degree polynomial, we need to convert a "global" check (over all the roots of unity) to a "local" check (at each root of unity).

Let's begin by rearranging the previous identity:

\begin{align*}
&\prod (\gamma - a'_i) \overset{?}{=} \prod (\gamma - b'_i) \\
\implies &\prod\frac{(\gamma - a'_i)}{(\gamma - b'_i)} \overset{?}{=} 1 \\
\end{align*}

More generally, we want to prove that the product over all terms of some vector $\vec{p}:$ is equal to some value
$$\prod f_0 \cdot f_1 \cdot f_{n-2} = p.$$
(In the example above, $p = 1$.) Note that the terms of $\vec{p}$ only go up to $n - 2$.

To express this in terms of low-degree local constraints, we introduce a "running product" vector $z$, constructed according to the following constraints:

\begin{cases}
z_0 &= 1, \texttt{ (boundary: initialisation)} \\
z_{i+1} &= f_{i} \cdot z_{i} \text{ for } 0 \leq i < n - 2, \texttt{ (accumulation)} \\
z_{n-1} &= p \texttt{ (boundary: final)}
\end{cases}

We observe that the $z$ vector "accumulates" terms from the original $p$ vector in a running product:

$$
\begin{array}{c|l}
f_i & z_i \\
\hline
f_0 & z_0 = 1\\
f_1 & z_1 = z_0 \cdot f_0 = f_0\\
f_2 & z_2 = z_1 \cdot f_1 = f_0 \cdot f_1\\
f_3 & z_3 = z_2 \cdot f_2 = f_0 \cdot f_1 \cdot f_2\\
\vdots & \vdots \\
f_{n-2} & z_{n-2} = f_0 \cdot f_1 \cdot \ldots \cdot f_{n-3} \\
        & z_{n-1} = f_0 \cdot f_1 \cdot \ldots \cdot f_{n-2} = p
\end{array}
$$

In order to check that the values of $z$ are indeed well-formed, we constrain it using the following polynomial identities:

\begin{cases}
L_0(X) \cdot (z(X) - 1) = 0, \texttt{ (boundary: initialisation)} \\
z(\omega X) = z(X) p(X) \texttt{ (accumulation)} \\
L_{n-1}(X) \cdot (z(X) - p) = 0, \texttt{ (boundary: final)} \\
\end{cases}

We can simplify the boundary constraints by adding a synthetic value $f_{n-1} = p^{-1}$. This makes the accumulation constraint "wrap around" on the last row:

$$
\begin{array}{c|l}
f_i & z_i \\
\hline
f_0 & z_0 = 1\\
f_1 & z_1 = z_0 \cdot f_0 = f_0\\
f_2 & z_2 = z_1 \cdot f_1 = f_0 \cdot f_1\\
f_3 & z_3 = z_2 \cdot f_2 = f_0 \cdot f_1 \cdot f_2\\
\vdots & \vdots \\
f_{n-2} & z_{n-2} = f_0 \cdot f_1 \cdot \ldots \cdot f_{n-3} \\
f_{n-1} = p^{-1} & z_{n-1} = f_0 \cdot f_1 \cdot \ldots \cdot f_{n-2} = p
\end{array}
$$

In other words, the expected next value $z_n = z_{n-1} \cdot f_{n-1} = p \cdot p^{-1} = 1$ is exactly constrained to be our initial $z_0 = 1$. This corresponds nicely with the behaviour of the roots of unity used to encode the row indices $\omega^n = 1 = \omega^0$.

Now, we are left only with the initialisation boundary constraint and the accumulation constraint. Using a random challenge $\alpha$ from the verifier, we combine these into a single polynomial identity that can be checked at a a random challenge:
$$L_0(X) \cdot (z(X) - 1) + \alpha \cdot (z(\omega X) - z(X) f(X)) = q(X) \cdot z_H(X).$$

Let's work through an example of the grand product argument. Working in the multiplicative group $\mathbb{F}_{13}$:

$$
H=(\omega^0=1,\omega^1=5,\omega^2=12,\omega^3=8)
$$

Suppose we want to prove the following product:

$$
p = f_0\cdot f_1 \cdot f_2 \cdot  f_3 = 2 \cdot 3 \cdot 2
$$

We construct the following table for the grand product:

$$
\begin{array}{c|c|l}
X_i & f_i & z_i \\
\hline
X_0 = \omega^0 = 1 & f_0 = 2 & z_0 = 1 \\
X_1 = \omega^1 = 5 & f_1 = 3 & z_1 = f_0 \cdot z_0 = 2 \\
X_2 = \omega^2 = 12 & f_2 = 2 & z_2 = f_1 \cdot z_1 = 6 \\
X_3 = \omega^3 = 8 & f_3=\frac{1}{p} = \frac{GF13(1)}{GF13(12)} = 12  &  z_3 = f_2 \cdot z_2 = p = 12 \\
X_4 = \omega^4 = 1 & & z_4 = f_3 \cdot z_3 = 144 \mod 13 = 1 \\
\end{array}
$$

Let's implement this in code below:

In [3]:
import numpy as np
import galois

GF13 = galois.GF(13)

# Given two 1-D arrays x and y
# Returns the Lagrange interpolating polynomial through the points (x, y).
def lagrange_poly_in_finite_field(x, y):
    return galois.lagrange_poly(GF13(x), GF13(y))

x_points = [1, 5, 12, 8]

L_0_values = [1, 0, 0, 0]
q_values = [2, 3, 2, 12]
r_values = [1, 2, 6, 12]
r_w_values = [2, 6, 12, 1]

assert q_values[3] == GF13(1)/ GF13(r_w_values[2])

# Do the interpolation, return polynomial
L_0_poly = lagrange_poly_in_finite_field(x_points, L_0_values)
q_poly = lagrange_poly_in_finite_field(x_points, q_values)
r_poly = lagrange_poly_in_finite_field(x_points, r_values)
r_w_poly = lagrange_poly_in_finite_field(x_points, r_w_values)

print("Output:")
print("Lagrange interpolating polynomial L_0_poly: ", L_0_poly)
print("Lagrange interpolating polynomial q_poly: ", q_poly)
print("Lagrange interpolating polynomial r_poly: ", r_poly)
print("Lagrange interpolating polynomial r_w_poly: ", r_w_poly)

# Vanishing polynomial: (X - 1)(X - 5)(X - 12)(X - 8)
z_H_poly = galois.Poly([1, 12], field = GF13) * galois.Poly([1, 8], field = GF13) * galois.Poly([1, 1], field = GF13) * galois.Poly([1, 5], field = GF13)

print("Vanishing polynomial z_H_poly: ", z_H_poly)

# This number should be be a random number
alpha = 10

combined_poly = L_0_poly * (r_poly - GF13(1)) + alpha * (r_poly * q_poly - r_w_poly)

# Quotient polynomial
quot_poly = combined_poly // z_H_poly

print("Quotient polynomial quot_poly: ", quot_poly)

assert combined_poly == quot_poly * z_H_poly, "Division has a remainder(should not have)"

# This number should be be a random number
zeta = 4

y_1 = combined_poly(zeta)
print("y_1: ", y_1)

y_2_poly = quot_poly * z_H_poly
y_2 = y_2_poly(zeta)
print("y_2: ", y_2)

assert y_1 == y_2, "Check failed"

print("Check successfully!")

Output:
Lagrange interpolating polynomial L_0_poly:  10x^3 + 10x^2 + 10x + 10
Lagrange interpolating polynomial q_poly:  5x^3 + 7x^2 + 8x + 8
Lagrange interpolating polynomial r_poly:  9x^3 + 8x^2 + 8x + 2
Lagrange interpolating polynomial r_w_poly:  7x^3 + 5x^2 + x + 2
Vanishing polynomial z_H_poly:  x^4 + 12
Quotient polynomial quot_poly:  7x^2 + 4x + 6
y_1:  6
y_2:  6
Check successfully!


## Putting it all together

Let's now apply the grand product argument to our original vectors $\vec{a}, \vec{b}$. Recall that the multiset equality identity is:

$$
\prod_{{i\in[n]}}\frac{(\gamma-p_i)}{(\gamma-q_i)}=1
$$
where:
\begin{align*}
p_i &= a_i + \beta\cdot i \\
q_i &= b_i + \beta\cdot i
\end{align*}

Substituting this into the multiset equality identity yields:
$$
\prod_{{i\in[n]}}\frac{(\gamma+a_i+\beta\cdot i)}{(\gamma+b_i+\beta\cdot \sigma(i))}=1,
$$
which we can check with the grand product argument.

### Example

Given：

$$
\vec{i}=(1,2,3,4),\quad \sigma = (2, 1, 4, 3)
$$
$$
\vec{a}=(2,4,2,1),\quad b = (4, 2, 1, 2)
$$

For the sake of illustration, assume that the random challenge $\beta = 2$，and compute $\vec{a'}$ and $\vec{b'}$：

$$
\begin{array}{|c|c|}
a'_i=(a_i+\beta\cdot i) & b_i'=(b + \beta\cdot \sigma(i)) \\
\hline
(2 + 2 \cdot 1) = 4 & (4 + 2 \cdot 2) = 8 \\
(4 + 2 \cdot 2) = 8 & (2 + 2 \cdot 1) = 4 \\
(2 + 2 \cdot 3) = 8 & (1 + 2 \cdot 4) = 9 \\
(1 + 2 \cdot 4) = 9 & (2 + 2 \cdot 3) = 8 \\
\end{array}
$$

We can now enforce the permutation between $\vec{a}$ and $\vec{b}$ using a multiset equality argument between $\vec{a'}$ and $\vec{b'}$:

$$
(4,8,8,9) = \vec{a'}\overset{?}{=} \vec{b'} = (8, 4, 9, 8)
$$

For the purpose of illustration, let the random challenge $\gamma = 3.$

Substituting the relevant values into the multiset equality identity:
$$
\prod_{{i\in[n]}}\frac{(\gamma+a_i+\beta\cdot i)}{(\gamma+b_i+\beta\cdot \sigma(i))}
= \prod_{{i\in[n]}}\frac{(\gamma+\vec{a'}(i))}{(\gamma+\vec{b'}(i))}
=1,
$$
we get:
$$
\frac{(3+4)(3+8)(3+8)(3+9)}{(3+8)(3+4)(3+9)(3+8)}
=\frac{7 \cdot 11 \cdot 11 \cdot 12}{11 \cdot 7 \cdot 12 \cdot 11}
=\frac{7}{11} \cdot \frac{11}{7} \cdot \frac{11}{12}\cdot \frac{12}{11}
=1 \pmod{13}
$$
In the finite field $\mathbb{F}_{13}$, this becomes:
$$
\frac{7}{11} \cdot \frac{11}{7} \cdot \frac{11}{12}\cdot \frac{12}{11}
=3 \cdot 9 \cdot 2 \cdot 7 
=1  \pmod{13}
$$

$$
\begin{array}{c|c|l}
X_i & f_i &  z_i \\
\hline
X_0 = \omega^0 = 1 & f_0 = 3 &  z_0 = 1  \\
X_1 = \omega^1 = 5 & f_1 = 9 &  z_1 = f_0 \cdot  z_0 = 3 \\
X_2 = \omega^2 = 12 & f_2 = 2 &  z_2 = f_1 \cdot  z_1 = 27 = 1  \pmod{13} \\
X_3 = \omega^3 = 8 & f_3 = 7  &  z_3 = f_2 \cdot  z_2 = 2 \\
X^4 = \omega^4 = 1 &          &  z_4 = f_3 \cdot  z_3 = 14 = 1  \pmod{13}
\end{array}
$$

We then check the well formation of $z$ using the grand product argument:
$$L_0(X) \cdot (z(X) - 1) + \alpha \cdot (z(\omega X) - z(X) f(X)) = q(X) \cdot z_H(X).$$

## Complete Permutation Argument

- **Public input**: Permutation relation $\sigma$
- **Private input**: Two vectors $\vec{a}$ and $\vec{b}$ 
- **Preprocessing**：Prover and Verifier construct commitments to the polynomial encodings of position vectors $[id(X)]$ and $[\sigma(X)]$

1. Prover commits to vectors and sends $[a(X)], [b(X)]$
2. Verifier sends challenges $\beta$ and $\gamma$
3. Prover constructs the running product vector $\vec{z}$ and encodes it as the polynomial $z(X)$; she sends a commitment $[z(X)]$
$$
\begin{split}
z_0 &= 1 \\
z_{i+1} &= z_i\cdot \frac{a_i+\beta\cdot i + \gamma}{b_i+\beta\cdot \sigma(i) + \gamma}
\end{split}
$$
4. Verifier sends the challenge $\alpha$
5. Prover constructs $f(X)$ and $q(X)$ as defined below, and sends the commitment $[q(X)]$
\begin{align*}
f(X) &= L_0(X)(z(X)-1) + \alpha\cdot (z(\omega\cdot X)(b(X)+\beta\cdot\sigma(X)+\gamma)-z(X)(a(X)+\beta\cdot id(X)+\gamma)) \\
q(X) &= \frac{f(X)}{z_H(X)}
\end{align*}

6. Verifier queries:
    - $[a(X)],[b(X)],[q(X)]$ at $X=\zeta$ obtaining the evaluations $a(\zeta)$， $b(\zeta)$， $q(\zeta)$
    - $[z(X)]$ at both $X=\{\zeta, \omega\zeta\}$, obtaining the evaluations $z(\zeta), z(\omega\cdot\zeta)$
    - $[\sigma(X)]$ and $[id(X)]$ at $X=\zeta$, obtaining the evaluations $id(\zeta)$ 与and$\sigma(\zeta)$
    
7. Verifier independently computes $z_H(\zeta)$, $L_0(\zeta)$

8. Verifier checks the grand product identity
$$
L_0(\zeta)(z(\zeta)-1) + \alpha\cdot (z(\omega\cdot \zeta)(b(\zeta)+\beta\cdot\sigma(\zeta)+\gamma)-z(\zeta)(a(\zeta)+\beta\cdot id(\zeta)+\gamma)) \overset{?}{=} q(\zeta)z_H(\zeta)
$$

See this in the implementation below:

In [2]:
import numpy as np
import galois

GF13 = galois.GF(13)

def get_div_num(a, b):
    return GF13(a) / GF13(b)

# Given two 1-D arrays x and y
# Returns the Lagrange interpolating polynomial through the points (x, y).
def lagrange_poly_in_finite_field(x, y):
    return galois.lagrange_poly(GF13(x), GF13(y))

x_points = [1, 5, 12, 8]

# Public inputs
i_vec = [1, 2, 3, 4]
sigma_vec = [2, 1, 4, 3]

# Private inputs
a_vec = [2, 4, 2, 1]
b_vec = [4, 2, 1, 2]

# Step 2: Verifier send challenge random number beta and gamma
# This number should be be a random number, here we just use a number as an example
beta = 2
gamma = 3

# Pre-Processing + Step 1 + Step 3.
# We do the steps together to get accumulator values r_values(z[i] values in step 3)

# Calculate q_values
q_values = []
for i in range(len(i_vec)):
  a_acc_num = a_vec[i] + beta * i_vec[i] + gamma
  b_acc_num = b_vec[i] + beta * sigma_vec[i] + gamma
  q_values.append(get_div_num(a_acc_num, b_acc_num))

# Calculate accumulator values r_values
r_values = [1]
for i in range(len(i_vec) - 1):
    r_values.append(r_values[-1] * q_values[i])

# Calculate r_w_values by shift r_values, the last value is 1
r_w_values = r_values[1:] + r_values[:1]

print("q_values: ", q_values)
print("r_values: ", r_values)
print("r_w_values: ", r_w_values)

assert np.array_equal(q_values, GF13([3, 9, 2, 7]))
assert np.array_equal(r_values, GF13([1, 3, 1, 2]))
assert np.array_equal(r_w_values, GF13([3, 1, 2, 1]))

L_0_values = [1, 0, 0, 0]

# Do the interpolation, return polynomial
L_0_poly = lagrange_poly_in_finite_field(x_points, L_0_values)
q_poly = lagrange_poly_in_finite_field(x_points, q_values)
r_poly = lagrange_poly_in_finite_field(x_points, r_values)
r_w_poly = lagrange_poly_in_finite_field(x_points, r_w_values)

print("Output:")
print("Lagrange interpolating polynomial L_0_poly: ", L_0_poly)
print("Lagrange interpolating polynomial q_poly: ", q_poly)
print("Lagrange interpolating polynomial r_poly: ", r_poly)
print("Lagrange interpolating polynomial r_w_poly: ", r_w_poly)

# Vanishing polynomial: (X - 1)(X - 5)(X - 12)(X - 8)
z_H_poly = galois.Poly([1, 12], field = GF13) * galois.Poly([1, 8], field = GF13) * galois.Poly([1, 1], field = GF13) * galois.Poly([1, 5], field = GF13)

print("Vanishing polynomial z_H_poly: ", z_H_poly)

# Step 4: Verifier send challenge random number alpha
# This number should be be a random number, here we just use a number as an example
alpha = 10

# Step 5: Calculate quotient polynomial
combined_poly = L_0_poly * (r_poly - GF13(1)) + alpha * (r_poly * q_poly - r_w_poly)

# Quotient polynomial
quot_poly = combined_poly // z_H_poly

print("Quotient polynomial quot_poly: ", quot_poly)

assert combined_poly == quot_poly * z_H_poly, "Division has a remainder(should not have)"

# Step 6
# This number should be be a random number
zeta = 4

y_1 = combined_poly(zeta)
print("y_1: ", y_1)

y_2_poly = quot_poly * z_H_poly
y_2 = y_2_poly(zeta)
print("y_2: ", y_2)

# Final step: Verify by verifier
assert y_1 == y_2, "Check failed"

print("Check successfully!")

q_values:  [GF(3, order=13), GF(9, order=13), GF(2, order=13), GF(7, order=13)]
r_values:  [1, GF(3, order=13), GF(1, order=13), GF(2, order=13)]
r_w_values:  [GF(3, order=13), GF(1, order=13), GF(2, order=13), 1]
Output:
Lagrange interpolating polynomial L_0_poly:  10x^3 + 10x^2 + 10x + 10
Lagrange interpolating polynomial q_poly:  6x^3 + 7x^2 + x + 2
Lagrange interpolating polynomial r_poly:  11x^3 + 9x^2 + 2x + 5
Lagrange interpolating polynomial r_w_poly:  10x^3 + 4x^2 + 10x + 5
Vanishing polynomial z_H_poly:  x^4 + 12
Quotient polynomial quot_poly:  3x^2 + 2x + 1
y_1:  1
y_2:  1
Check successfully!


## References:

- [WIP] Copy constraint for arbitrary number of wires. https://hackmd.io/CfFCbA0TTJ6X08vHg0-9_g
- Alin Tomescu. Feist-Khovratovich technique for computing KZG proofs fast. https://alinush.github.io/2021/06/17/Feist-Khovratovich-technique-for-computing-KZG-proofs-fast.html#fn:FK20
- Ariel Gabizon. Multiset checks in PLONK and Plookup. https://hackmd.io/@arielg/ByFgSDA7D
